In [1]:
import json_lines
from langdetect import detect
from langdetect import detect_langs
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pandas as pd

lemmatizer = WordNetLemmatizer()
REMOVEABLE_CHARS = "»<>123456789\/&()?:!.,;'´"
# ALL_TAGS = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
#  'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
#  'WP$', 'WRB']
ACCEPTABLE_TAGS = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
MAX_FEATURES = 50
MIN_ENGLISH = 0.9999
LIMIT_INPUT_ROWS = 1000 # to stop run times hindering development

def get_wordnet_pos(tag): # Return a char, based on input tag, that is used as a param in lemmatisation for enhanced results
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [2]:
# READ IN INFO
X= []; y= []; z= [];
with open ('reviews.json', 'rb') as f:
    for i, item in enumerate(json_lines.reader(f)):
        if i <= LIMIT_INPUT_ROWS:
            try:
                langs = detect_langs(item['text'])
            except:
                pass
            for i, lang in enumerate(langs):
                if str(lang)[0:2] == 'en':
                    if float(str(lang)[3:]) > MIN_ENGLISH:
                        X.append(item['text'])
                        y.append(item['voted_up'])
                        z.append(item['early_access'])

In [3]:
for i, item in enumerate(X):
    print( "X[", str(i), "]: ", str(X[i]) )
    print( "y[", str(i), "]: ", str(y[i]) )
    print( "z[", str(i), "]: ", str(z[i]) )
    print( "\n" )

X[ 0 ]:  neat little game. play if your staple fps game's server is down or sth. maps and weapon distribution system needs rework.
y[ 0 ]:  True
z[ 0 ]:  True


X[ 1 ]:  Kinda decent f2p mmorpg that has no class limitations(hooray!)(and that game where i drained too much hours...) Community: In PvE, you may encounter friendly ppl(i managed to even encounter person that i love :o), while in PvP mostly complainers(those who complain about AT&T releasing too much of 1st in battle xD) or persons that make you turn into complainer.(As always!) Main review part: Graphics: Everything looks kinda pretty, but performance is not what expected for java and openGL...(8/10) Gameplay: SK features huge choice of equipment, which adds variety to the gear and gameplay itself(like you can use guns, or bombs, or swords.. or 2 of those... or all 3 xD) But because this game features grinding in big quantities you  may get bored(or may not if you do co-op with your friend(s))(8/10) Controls: In order to rek

z[ 114 ]:  False


X[ 115 ]:  the game is kinda  ba dum tss
y[ 115 ]:  True
z[ 115 ]:  True


X[ 116 ]:  love this game. very cute. very funny. very fun to play. simple, sweet and stylish. recommend. a short game but a great time playing it. quite addictive.
y[ 116 ]:  True
z[ 116 ]:  False


X[ 117 ]:  Best game to play in "SOLO" mode xdxdd._. cuz its solo mode...
y[ 117 ]:  True
z[ 117 ]:  False


X[ 118 ]:  Short comedy based on a good series worth the 3$!!!  Also 10 pages worth of saves for less than a 3 hour game XD
y[ 118 ]:  True
z[ 118 ]:  False


X[ 119 ]:  +Best mmo i have ever played. +Worth to buy the dlc's. +A must have for a The elder scrolls lover. +Still fun playing the full game alone (but still better with friends). +Very good PvP. +Very good skill system. +Beautiful graphics. -It needs a bit of time to get into the game (the start sucks a little bit). -The server are often down. -If You are high lvl the quest are getting way too easy. -Many buggs.
y[ 119 ]:  True
z[ 

z[ 234 ]:  False


X[ 235 ]:  It's aight
y[ 235 ]:  True
z[ 235 ]:  False


X[ 236 ]:  best story game marvel
y[ 236 ]:  True
z[ 236 ]:  False


X[ 237 ]:  Fantastic but very hard game! I realy like the graphics of this game and the difficulty. No run is the same you find always every run some other weapons and gadgets.
y[ 237 ]:  True
z[ 237 ]:  False


X[ 238 ]:  Metro: Exodus. This game! i played the other games before, as a release titled by time and the enhanced editions that were on discount on steam, i played the games before having to be annoyed by the fact that it is not an open world and you cant do what you want, but Exodus... well this game is a state of the art! this is one of my favorites for the last month and i cant stop playing it, i am a strong survivor playing on hardcore mode, i have the machine to deliver the goods for this beautiful masterpiece!  and i am shocked by the way the game is delivering exactly what was promised, i am happy to say i got what i payed for,

In [4]:
# FILTER TEXT
for i, text in enumerate(X):
    text_words = nltk.word_tokenize(text.lower())
    tags = nltk.pos_tag(text_words)
    #print(tags)
    new_text= [];
    for j, word in enumerate(text_words):
        if tags[j][1] in ACCEPTABLE_TAGS:
            for char in word:
                if char in REMOVEABLE_CHARS:
                    break
            else:
                wordnet_pos = get_wordnet_pos(tags[j][1])
                if wordnet_pos != '':
                    new_text.append(lemmatizer.lemmatize(word, wordnet_pos))
                    #print("word: ", word, ". wordnet_pos: ", wordnet_pos, "lemmad: ",lemmatizer.lemmatize(word, wordnet_pos),"\n")
                continue
    X[i] = new_text

In [5]:
# MOST FREQ WORDS AS FEATURES
word_counts = {};
for text in X:
    for word in text:
        if word in word_counts:
            word_counts[word]+= 1
        else:
            word_counts[word]= 1
word_counts = {k: v for k, v in sorted(word_counts.items(), key=lambda item: item[1], reverse=True)}
trimd_word_counts = {}
if len(word_counts) > MAX_FEATURES:
    for i, entry in enumerate(word_counts):
        if i >= MAX_FEATURES:
            break
        trimd_word_counts[entry] = word_counts[entry]
word_counts = trimd_word_counts
print(word_counts)

{'be': 806, 'game': 594, 'i': 383, 'have': 217, 'do': 152, 'play': 124, 'get': 116, 'good': 94, 'not': 90, 'so': 85, 'make': 83, 'story': 80, 'more': 78, 'time': 76, 'fun': 73, 'just': 71, 'well': 62, 'great': 60, 'really': 58, 'best': 57, 'also': 52, 'very': 51, 'love': 50, 'go': 50, 'still': 50, 'much': 49, 'other': 49, 'thing': 48, 'character': 47, 'player': 44, 'new': 42, 'weapon': 40, 'use': 39, 'only': 39, 'people': 39, 'need': 38, 'even': 38, 'friend': 38, 'want': 37, 'buy': 37, 'way': 37, 'hour': 36, 'say': 36, 'first': 36, 'up': 35, 'think': 35, 'take': 34, 'many': 33, 'then': 32, 'mode': 32}


In [6]:
# CREATE 1-HOT ENCODINGS
one_hots = [];
for text in X:
    one_hot = [];
    for i in range(MAX_FEATURES):
        one_hot.append(0)
    for word in text:
        if word in word_counts:
            one_hot[list(word_counts).index(word)]+= 1
    one_hots.append(one_hot)
    
print(one_hots)

[[1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4, 7, 3, 2, 2, 0, 4, 1, 4, 1, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 4, 0, 0, 0, 0, 2, 0, 1, 1, 0, 1, 2, 2, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [5, 4, 3, 5, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 3, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0], [1, 2, 3, 0, 0, 0, 1, 0, 1, 1, 2, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,

In [7]:
# # CONVERT TO PANDAS
# d = {'Text':X, 'Voted up':y, 'Early access':z}
# df = pd.DataFrame(d)
# df